# Train an object detection model using Ground Truth labels
At this stage, you have fully labeled your dataset and you can train a machine learning model to perform object detection. You'll do so using the **augmented manifest** output of your labeling job - no additional file translation or manipulation required! For a more complete description of the augmented manifest, see our other [example notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/ground_truth_labeling_jobs/object_detection_augmented_manifest_training/object_detection_augmented_manifest_training.ipynb).

**NOTE:** Object detection is a complex task, and training neural networks to high accuracy requires large datasets and careful hyperparameter tuning. The following cells illustrate how to train a neural network using a Ground Truth output augmented manifest, and how to interpret the results. However, you shouldn't expect a network trained on 10 or 1000 images to do a great job on unseen images!

## Set Up

First import the required libs. Use the `%` magic directive to enable matplotlib inline plots.

In [1]:
%matplotlib inline

import json, boto3, sagemaker, re, time, sys, glob, os
from time import gmtime, strftime
import numpy as np
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role
from hint import hint
from helper import training_status, visualize_detection

First we need to unzip an archive of previously generated ground truth annotations and their related images. The where generated and placed in to S3 by Ground Truth, but for simplicity in this lab they have been archived. 

In [2]:
!wget  -q http://d8mrh1kj01ho9.cloudfront.net/data/object_detection_data.zip
!unzip -qn object_detection_data.zip

## Prepare Data

First lets determine some constant values that we need create paths in S3 and locally:

In [ ]:
LOCAL_ROOT_PATH = './object_detection_data'
print('Location of lab data locally is {}'.format(LOCAL_ROOT_PATH))

# function to get full path for manifest file (used for s3 or local)
def get_manifest_path(root_path, manifest_type):
    return '{}/ground_truth_output/all-bird-labels/manifests/output/{}.manifest'.format(root_path, manifest_type)

output_manifest_path = get_manifest_path(LOCAL_ROOT_PATH, 'output')
print('Manifest file is {}'.format(output_manifest_path))

ACCOUNT_ID = boto3.client("sts").get_caller_identity()["Account"]
REGION = boto3.session.Session().region_name
USER = 'default' # change if multiple users in this account will run this lab 
BUCKET = 'lab-{}'.format(ACCOUNT_ID)
S3_ROOT_PATH = 's3://{}/users/{}/object_detection'.format(BUCKET, USER)
print('Lab data will be synced with {} in region {}'.format(S3_ROOT_PATH, REGION))

Lets take a look at what the ground truth produced output manifest file looks like. The command line below pretty prints the first line in the annotation file.

In [ ]:
!head -1 $output_manifest_path | jq .

Now we willl split the augmented manifest into a training set and a validation set using an 80/20 split and save the results to files that the model will use during training. To do this, the output manifest is read to get a list of all the images that are labeled. Since we have copied data from the original location in S3 to a location for this account, we need to transform the s3 paths (source ref) within the manifest as well.

In [ ]:
# read the output manifest, and modify the image path to work with new s3 location
manifest = []
with open(output_manifest_path, 'r') as f:
    for line in f.readlines():
        manifest_item = json.loads(line.strip())
        image_file = manifest_item['source-ref'].split('images/')[-1]
        manifest_item['source-ref'] = '{}/images/{}'.format(S3_ROOT_PATH, image_file)
        manifest.append(manifest_item)
        
dataset_size = len(manifest)
print('Read {} lines from output manifest {}'.format(dataset_size, output_manifest_path))

# shuffle output
# it is good practice to shuffle data so 
# annotations from the same annotator are not all together 
# for training or validation
np.random.shuffle(manifest)
 
# now split the data into train and test data
split_index = round(dataset_size*0.8)
train_data = manifest[:split_index]
validation_data = manifest[split_index:]

# create the training manifest
train_manifest_path = get_manifest_path(LOCAL_ROOT_PATH, 'train')
num_training_samples = 0
with open(train_manifest_path, 'w') as f:
    for line in train_data:
        f.write(json.dumps(line) + '\n')
        num_training_samples += 1
print('Created training manifest {} \n...with {} samples'.format(train_manifest_path, num_training_samples))

# this a function to get full path for validation manifest (used for s3 or local)
get_train_manifest = lambda root:'{}/ground_truth_output/all-bird-labels/manifests/output/train.manifest'.format(root)

# write the validation manifest locally
validation_manifest_path = get_manifest_path(LOCAL_ROOT_PATH, 'validation')
num_validation_samples = 0
with open(validation_manifest_path, 'w') as f:
    for line in validation_data:
        f.write(json.dumps(line) + '\n')
        num_validation_samples += 1
print('Created validation manifest {} \n...with {} samples'.format(validation_manifest_path, num_validation_samples))

Next, we'll sync all the images, and manifest files with the previously defined S3 bucket so that they can be used in the training job. The code below creates the S3 bucket if it doesn't already exist, and then syncs the local files to the S3 location. We need all our images and annotations in S3 for SageMaker training.

In [ ]:
!aws s3api create-bucket --bucket $BUCKET --region $REGION --create-bucket-configuration LocationConstraint=$REGION
!aws s3 sync $LOCAL_ROOT_PATH $S3_ROOT_PATH --quiet

## Initial Training
Now that you are done with all the setup that is needed, you are ready to train your object detector. To begin, create a SageMaker Estimator object. This estimator will launch the training job. [See the SageMaker SDK docs for what parameters are needed.](https://sagemaker.readthedocs.io/en/stable/estimators.html)

Use a single 'ml.p2.xlarge' instance for training, with an input mode of pipe, volume size of 50, and max training runs of 10,000. The output path for the model artifacts is the previously defined model path. Use the hint button if you run into trouble.

In [ ]:
hint('estimator')

In [ ]:
# get required objects and parameters
role = get_execution_role()
training_image = get_image_uri(REGION, 'object-detection', repo_version='latest')
session = sagemaker.Session()
model_path = '{}/models'.format(S3_ROOT_PATH)

# create an estimator
estimator = # TODO

print('Created estimator. All artifacts will be generated in {}'.format(model_path))

The object detection algorithm at its core is the Single-Shot Multi-Box detection algorithm (SSD). This algorithm uses a base_network, which is typically a VGG or a ResNet. The Amazon SageMaker object detection algorithm supports VGG-16 and ResNet-50 now. It also has a lot of options for hyperparameters that help configure the training job. The next step in your training, is to setup these hyperparameters and data channels for training the model. Consider the following example definition of hyperparameters. See the SageMaker Object Detection documentation for more details on the hyperparameters.

One of the hyperparameters here for instance is the epochs. This defines how many passes of the dataset you iterate over and determines that training time of the algorithm. In this example, you train the model for 5 epochs to generate a basic model for the PASCAL VOC dataset. [See the AWS docs for more info on the available hyperparameters for object detection](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-tuning.html)

In [ ]:
# These hyperparameters are at the user's discretion and are beyond the scope of this demo.
hyperparameters = {  
            "base_network": "resnet-50",
            "use_pretrained_model": "1",
            "num_classes": "1",
            "mini_batch_size": 1,
            "epochs": 1,
            "learning_rate": "0.001",
            "lr_scheduler_step": "",
            "lr_scheduler_factor": "0.1",
            "optimizer": "sgd",
            "momentum": "0.9",
            "weight_decay": "0.0005",
            "overlap_threshold": "0.5",
            "nms_threshold": "0.45",
            "image_shape": "300",
            "label_width": "350",
            "num_training_samples": str(num_training_samples)
        }

estimator.set_hyperparameters(**hyperparameters)
print('set hyperparameters')

Now that the hyperparameters are setup, prepare the handshake between your data channels and the algorithm. To do this, you need to create the sagemaker.session.s3_input objects from your data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [ ]:
train_manifest = get_manifest_path(S3_ROOT_PATH, 'train')
validation_manifest = get_manifest_path(S3_ROOT_PATH, 'validation')

def create_data_input(uri):
    return sagemaker.session.s3_input(
        uri, 
        s3_data_type='AugmentedManifestFile',
        distribution='FullyReplicated', 
        content_type='application/x-recordio',
        record_wrapping='RecordIO',
        attribute_names = ['source-ref', 'all-bird-labels']
    )

data_channels = {
    'train': create_data_input(train_manifest), 
    'validation': create_data_input(validation_manifest)
}
print('created data channels')

In [ ]:
print (train_manifest, validation_manifest)

Now that you have the Estimator object, you have to set the hyperparameters for the Estimator and  have the data channels linked with the algorithm. The only remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instances that you requested while creating the Estimator classes are provisioned and are setup with the appropriate libraries. Then, the data from your channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take time, depending on the size of the data. Therefore it might be a few minutes before you start getting data logs for your training jobs. The data logs will also print out Mean Average Precision (mAP) on the validation data, among other losses, for every run of the dataset once or one epoch. This metric is a proxy for the quality of the algorithm.

Use the  defined data channels and job name as inputs, and be sure to set wait is False and logs is True.

In [ ]:
hint('fit')

In [ ]:
# create unique job name
job_name = '{}-od-{}'.format(USER, time.strftime('%Y%m%d%H%M%S', time.gmtime()))

# start training
# TODO

print('training job {} launched'.format(job_name))

To check the progess of the training job, you can repeatedly evaluate the following cell. When the training job status reads `'Completed'`, move on to the next part of the tutorial.

In [ ]:
training_status(job_name)

## Hosting

Once the training is done, you can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow you to make predictions (or inference) from the model. Note that you don't have to host on the same instance (or type of instance) that you used to train. Training is a prolonged and compute heavy job that require a different of compute and memory requirements that hosting typically do not. You can choose any type of instance you want to host the model. In your case you chose the `ml.p3.2xlarge` instance to train, but you choose to host the model on the less expensive cpu instance, `ml.m4.xlarge`. Only use a single instance type for development, but a minimum of 2 (whihc will deploy to two AZ's) for production.

In [ ]:
hint('hosting')

In [ ]:
object_detector = # TODO

## Inference

Now that the trained model is deployed at an endpoint that is up-and-running, you can use this endpoint for inference. To do this, you will download the previously created validation set.

In [ ]:
file_names = []
gt_bboxes = []
s3 = boto3.resource('s3')

# loop through all validation data and gather the local file path and bounding boxes from the annotation
for file_data in validation_data:
    s3_file_path = file_data['source-ref'].split('images/')[-1]
    local_file_path = '{}/images/{}'.format(LOCAL_ROOT_PATH, s3_file_path)

    file_names.append(local_file_path)
    
    if 'annotations' in file_data['all-bird-labels']:
        annotations = file_data['all-bird-labels']['annotations']
        bboxes = []
        for a in annotations:
            bboxes.append([int(a['left']), int(a['top']), int(a['width']), int(a['height'])]) 
        gt_bboxes.append(bboxes)

Now, use your endpoint to try to detect objects within this image. Since the image is jpeg, you use the appropriate content_type to run the prediction job. The endpoint returns a JSON file that you can simply load and peek into.

Call predict on the object detector that is deployes, and convert the string returned to json. 

In [ ]:
hint('infer')

In [ ]:
count = 0
for file_name in file_names:
    with open(file_name, 'rb') as image:
        f = image.read()
        b = bytearray(f)
        
        # skip large images due to timeout issue
        if len(b) > 800000:
            continue
         
        # get inference
        results = # TODO
        detections = # TODO
        
        count += 1
        if count > 20:
            break

The results are in a format that is similar to the .lst format with an addition of a confidence score for each detected object. The format of the output can be represented as `[class_index, confidence_score, xmin, ymin, xmax, ymax]`. Typically, you don't consider low-confidence predictions.

You have provided additional script to easily visualize the detection outputs. You can visualize the high-confidence predictions with bounding box by filtering out low-confidence detections using the script below:

In [ ]:
object_categories = ['bird']

# Setting a threshold 0.20 will only plot 
# detection results that have a confidence score greater than 0.20.
# adjust this value until you only see a few colored bounding boxes
threshold = 0.33

# Visualize the detections
max_to_display = 10
for file_name, bboxes in zip(file_names,gt_bboxes):
    visualize_detection(file_name, detections['prediction'], object_categories, threshold, bboxes)
    max_to_display -= 1
    if max_to_display == 0:
        break

No suprise, a model trained on 10 images is not doing so great compared to the human labels. Also, for the sake of this quick training, you trained the model with only one epoch. To achieve better detection results, you could try to tune the hyperparameters and train the model for more epochs with a larger dataset. 

### Delete the Endpoint

Having an endpoint running will incur some costs. Therefore as a clean-up job, you should delete the endpoint.

In [ ]:
sagemaker.Session().delete_endpoint(object_detector.endpoint)